In [1]:
from BCBio import GFF
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord

In [2]:
genomeFile = 'alignment/genomeBuild/NZ_CP022438_S.peucetius_subsp.caesius_ATCC_27952_chromosome.gb'
g = SeqIO.read(genomeFile, 'genbank')

In [5]:
type(g)

Bio.SeqRecord.SeqRecord

TODO: we can extract more feature types like rRNA and then filter after the counting has been done.
Try the group features function to in featureCounts program (meta-feature level summarization)

In [11]:
# extract only genes to write to gff file
onlyGenesRec = SeqRecord(g.seq, id=g.id, name='NZ_CP022438')
genes = []
for feat in g.features:
    if feat.type == 'gene':
        genes.append(feat)
onlyGenesRec.features.extend(genes)

In [12]:
with open('alignment/genomeBuild/NZ_CP022438_S.peucetius_subsp.caesius_ATCC_27952_chromosome.gff', 'w') as f:
    GFF.write([onlyGenesRec], f)

In [17]:
import BCBio

In [18]:
dir(BCBio)

['GFF',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__']

Convert GFF to GTF

locus_tag=CGZ69_RS00005;old_locus_tag=CGZ69_00005

locus_tag "CGZ69_RS00005";old_locus_tag "CGZ69_00005"

In [23]:
with open('alignment/genomeBuild/NZ_CP022438_S.peucetius_subsp.caesius_ATCC_27952_chromosome.gff', 'r') as s:
    with open('alignment/genomeBuild/NZ_CP022438_S.peucetius_subsp.caesius_ATCC_27952_chromosome.gtf', 'w') as t:
        for l in s.readlines():
            if l.startswith('##'):
                continue
            l = l.strip()
            elements = l.split('\t')
            rawAttributes = [e.split('=') for e in elements[-1].split(';')]
            gtfAttrs = ';'.join([f'{a} "{v}"' for a,v in rawAttributes])
            t.write('\t'.join(elements[:-1]+[gtfAttrs])+'\n')
            

Count rRNA

In [3]:
rrnas = []
for feat in g.features:
    if feat.type == 'rRNA':
        rrnas.append(feat.qualifiers['locus_tag'][0])
for r in rrnas:
    print(r)

CGZ69_RS05490
CGZ69_RS05495
CGZ69_RS05500
CGZ69_RS07735
CGZ69_RS07740
CGZ69_RS07745
CGZ69_RS13750
CGZ69_RS13755
CGZ69_RS13760
CGZ69_RS17095
CGZ69_RS17100
CGZ69_RS17105
CGZ69_RS20320
CGZ69_RS20325
CGZ69_RS20330
CGZ69_RS26990
CGZ69_RS26995
CGZ69_RS27000


In [4]:
import os
os.listdir('genecounts')[:5]


['180087_1_WT_1d_S1_L001_NCBI.txt.summary',
 '180087_1_WT_1d_S1_L001_NCBI.txt',
 '180087_1_WT_1d_S1_L002_NCBI.txt',
 '180087_1_WT_1d_S1_L002_NCBI.txt.summary',
 '180087_2_WT_2d_S11_L001_NCBI.txt']

In [5]:
# test
f = os.listdir('genecounts')[1]
f

'180087_1_WT_1d_S1_L001_NCBI.txt'

In [6]:
f = os.path.join('genecounts', f)
f

'genecounts/180087_1_WT_1d_S1_L001_NCBI.txt'

In [9]:
import pandas as pd
ct = pd.read_csv(f, sep='\t', comment='#', header=0, index_col=0)
ct.head()

,Chr,Start,End,Strand,Length,alignment/NCBI_genome/180087_1_WT_1d_S1_L001_NCBI.sam
Geneid,,,,,,
CGZ69_RS00005,NZ_CP022438.1,628,1047,-,420,0
CGZ69_RS00010,NZ_CP022438.1,1282,1740,+,459,4
CGZ69_RS00015,NZ_CP022438.1,2706,2909,+,204,1
CGZ69_RS00020,NZ_CP022438.1,3132,3335,-,204,0
CGZ69_RS00030,NZ_CP022438.1,4021,4309,+,289,0


In [12]:
ct.index.map(lambda x: x not in rrnas)

Index([True, True, True, True, True, True, True, True, True, True,
       ...
       True, True, True, True, True, True, True, True, True, True],
      dtype='object', name='Geneid', length=7129)

In [13]:
ct.shape

(7129, 6)

In [14]:
ct[ct.index.map(lambda x: x not in rrnas)].shape

(7111, 6)

In [15]:
newct = ct[ct.index.map(lambda x: x not in rrnas)]
newct.head()

,Chr,Start,End,Strand,Length,alignment/NCBI_genome/180087_1_WT_1d_S1_L001_NCBI.sam
Geneid,,,,,,
CGZ69_RS00005,NZ_CP022438.1,628,1047,-,420,0
CGZ69_RS00010,NZ_CP022438.1,1282,1740,+,459,4
CGZ69_RS00015,NZ_CP022438.1,2706,2909,+,204,1
CGZ69_RS00020,NZ_CP022438.1,3132,3335,-,204,0
CGZ69_RS00030,NZ_CP022438.1,4021,4309,+,289,0


In [19]:
newct.iloc[:,5].sum()

87698

In [20]:
ct.iloc[:,5].sum()

8916438

## Now remove all rrnas from the table

In [26]:
# os.mkdir('genecounts_no_rRNA')
for f in os.listdir('genecounts'):
    if f.endswith('.summary'):
        continue
    try:
        ct = pd.read_csv(f'genecounts/{f}', sep='\t', comment='#', header=0, index_col=0)
    except:
        print(f)
        continue
    newct = ct[ct.index.map(lambda x: x not in rrnas)]
    newct.columns = [c.replace('alignment/NCBI_genome/','') for c in newct.columns]
    newct.to_csv(os.path.join('genecounts_no_rRNA', f'{os.path.splitext(f)[0]}_no_rRNA.txt'), sep='\t')
    

.ipynb_checkpoints
